In [41]:
import pandas as pd
import numpy as np

# Feature engineering
import unicodedata
from nltk import ngrams

## Data Cleaning

### Cleaning Indonesian Dataset

We have two Indonesian datasets, so let's compare them:

In [2]:
# Inspecting first Indonesian dataset
df_indo_1 = pd.read_excel('name_data/exigerData/EXGR_Indonesian names.xlsx')
df_indo_1.head()

,Unnamed: 0,fullname,Family name,Given name
0,http://www.wikidata.org/entity/Q7313790,Bertrand Antolin,NaN,NaN
1,http://www.wikidata.org/entity/Q15138877,Ray Rizal,NaN,NaN
2,http://www.wikidata.org/entity/Q17411237,Samsuridjal Djauzi,NaN,NaN
3,http://www.wikidata.org/entity/Q12497619,Max Arifin,NaN,NaN
4,http://www.wikidata.org/entity/Q7475963,Donita,NaN,NaN


In [3]:
# Inspecting second Indonesian dataset
df_indo_2 = pd.read_excel('name_data/exigerData/EXGR_Indonesian names-2.xlsx')
df_indo_2.head()

,Unnamed: 0,fullname,Family name,Given name,Designation,Comment,Unnamed: 6,Unnamed: 7
0,http://www.wikidata.org/entity/Q7475963,Donita,NaN,Donita,NaN,It is common for Indonesians to have only 1 gi...,NaN,1
1,http://www.wikidata.org/entity/Q7645143,Supriyadi,NaN,Supriyadi,NaN,NaN,NaN,1
2,http://www.wikidata.org/entity/Q7844727,Triyaningsih,NaN,Triyaningsih,NaN,NaN,NaN,1
3,http://www.wikidata.org/entity/Q12515781,Soerjadi,NaN,Soerjadi,NaN,NaN,NaN,1
4,http://www.wikidata.org/entity/Q12523244,Undunsyah,NaN,Undunsyah,NaN,NaN,NaN,1


Are the names in the datasets the same?

In [4]:
try:
    pd.testing.assert_series_equal(df_indo_1['fullname'], df_indo_2['fullname'], check_like = True)
except:
    print("Names are not the same")

Names are not the same


We will investigate this further later. For now, let's look at the comments in `df_indo_2`:

In [5]:
print(df_indo_2['Comment'].unique()[:5])
print('\nNumber of names with comments:', len(df_indo_2['Comment'].unique()))
df_indo_2.shape

['It is common for Indonesians to have only 1 given name without family name.'
 nan 'Real name: Mike Lucock'
 'This is a Javanese ancient name. The name is not used anymore. This person lived in 1028 - 1035.'
 'Royal family name of Java identified by the designation name.']

Number of names with comments: 34


(21727, 8)

Since these names seem uncommon exceptions and there are only a few of them, we will remove them to make the data cleaner:

In [6]:
df_indo_2 = df_indo_2[pd.isnull(df_indo_2['Comment'])]
df_indo_2.reset_index()
print(df_indo_2.shape)
df_indo_2['Comment'].unique()

(21689, 8)


array([nan], dtype=object)

Making dataframes consist only of the `fullname` column:

In [7]:
df_indo_1 = pd.DataFrame(df_indo_1['fullname'])
df_indo_2 = pd.DataFrame(df_indo_2['fullname'])
print('df_indo_1 shape:', df_indo_1.shape)
print('df_indo_2 shape:', df_indo_2.shape)

df_indo_1 shape: (21730, 1)
df_indo_2 shape: (21689, 1)


Combining both dataframes into `df_indo`:

In [8]:
# Combining datasets
df_indo = pd.concat([df_indo_1, df_indo_2], ignore_index = True)
print('Shape of combined data:', df_indo.shape)
df_indo.head()

Shape of combined data: (43419, 1)


,fullname
0,Bertrand Antolin
1,Ray Rizal
2,Samsuridjal Djauzi
3,Max Arifin
4,Donita


Looking for duplicate entries:

In [9]:
print(len(df_indo[df_indo.duplicated()]))

32085


Since around 3/4 of the combined data consists of duplicate entries, the datasets are very similar. Let's just use `df_indo_2` since we removed the names with comments from it:

In [10]:
df_indo = df_indo_2
print(df_indo.shape)
df_indo.head()

(21689, 1)


,fullname
1,Supriyadi
2,Triyaningsih
3,Soerjadi
4,Undunsyah
5,Soeripto


Are there duplicates?

In [11]:
len(df_indo[df_indo.duplicated()])

10376

Let's drop them:

In [12]:
# Dropping duplicates
df_indo.drop_duplicates(inplace = True, ignore_index = True)
print(np.any(df_indo.duplicated()))
df_indo.shape

False


(11313, 1)

Let's see if there are any null entries for `fullname`:

In [13]:
np.any(df_indo['fullname'].isnull())

False

What about entries that aren't alphanumeric aside from spaces?

In [14]:
non_alnum_names_indo = [name for name in df_indo['fullname'] if not str.isalnum(name.replace(' ', ''))]
print(len(non_alnum_names_indo))
non_alnum_names_indo[-10:]

590


['I. B. Rai Dharmawijaya Mantra',
 'Mansoer Daoed Dt. Palimo Kayo',
 'Habib Anis bin Alwi al-Habsyi',
 'Abu Bakar (bupati Bandung Barat)',
 'Muhammad Arsyad (pemain sepak bola)',
 'Muhammad Hamzah (personil The Fly)',
 'A. A. Ngurah Oka Ratmadi',
 'Firmansyah (pemain sepak bola kelahiran 1995)',
 'Mayjen TNI (Purn) Thomas Albert Umboh',
 'Prof. Dr. Ir. Antonius Suwanto, M.Sc']

Some names have parentheses. Let's see exactly how many:

In [15]:
paren_names_indo = [name for name in df_indo['fullname'] if name.find('(') != -1]
print(len(paren_names_indo))
paren_names_indo[:5]

67


['Basuki (pelawak)',
 'Mulyadi (politisi)',
 'Fatahillah (politisi)',
 'Trinity (penulis)',
 'Mentari (aktris)']

Removing the parentheses from the names and seeing how many names are non-alphanumeric:

In [16]:
df_indo['fullname'] = df_indo['fullname'].apply(lambda name: name if name.find('(') == -1 else name[:name.find('(') - 1])
non_alnum_names_indo = [name for name in df_indo['fullname'] if not str.isalnum(name.replace(' ', ''))]
print(len(non_alnum_names_indo))

527


The number of names left isn't exactly 590 - 67 = 523 because some names with parentheses had other punctuation in it. Let's double check that there are no names with parentheses in the `fullname` column:

In [17]:
print([name for name in df_indo['fullname'] if name.find('(') != -1])
print([name for name in df_indo['fullname'] if name.find(')') != -1])

[]
[]


It's possible that we have some duplicate names after removing parentheses, so let's check for that:

In [18]:
print(len(df_indo[df_indo.duplicated()]))
print(df_indo.shape)
df_indo[df_indo.duplicated()]

63
(11313, 1)


,fullname
1676,Basuki
1810,Mulyadi
2716,Fatahillah
3080,Trinity
4798,Mentari
...,...
11285,Abu Bakar
11286,Muhammad Arsyad
11288,Muhammad Hamzah
11300,Firmansyah


Removing leading and trailing whitespace from names just in case:

In [19]:
df_indo['fullname'] = df_indo['fullname'].apply(str.strip)

Removing duplicates again:

In [20]:
df_indo.drop_duplicates(inplace = True, ignore_index = True)
print(np.any(df_indo.duplicated()))
df_indo.shape

False


(11246, 1)

### Cleaning Malay Dataset

Let's look at the Malay dataset now:

In [21]:
df_malay = pd.read_excel('name_data/exigerData/EXGR_Malay names.xlsx')
df_malay.head()

,Unnamed: 0,fullname,Family name,Given name
0,http://www.wikidata.org/entity/Q4769705,Annuar Rapaee,NaN,NaN
1,http://www.wikidata.org/entity/Q31186972,Tash Yong,NaN,NaN
2,http://www.wikidata.org/entity/Q468519,Fatmawati,NaN,NaN
3,http://www.wikidata.org/entity/Q4736793,Alto Linus,NaN,NaN
4,http://www.wikidata.org/entity/Q28837179,Mohamad Izzat Abdul Halil,NaN,NaN


Restricting the dataframe to just the `fullname` column:

In [22]:
df_malay = pd.DataFrame(df_malay['fullname'])
df_malay.head()

,fullname
0,Annuar Rapaee
1,Tash Yong
2,Fatmawati
3,Alto Linus
4,Mohamad Izzat Abdul Halil


Handling duplicate values:

In [23]:
# Checking for duplicate entries
print(len(df_malay[df_malay.duplicated()]))
df_malay.shape

2016


(4930, 1)

In [24]:
# Removing duplicate entries
df_malay.drop_duplicates(inplace = True, ignore_index = True)
print(np.any(df_malay.duplicated()))
df_malay.shape

False


(2914, 1)

Are there any null entries in `fullname`?

In [25]:
print(np.any(df_malay['fullname'].isnull()))

False


Are any names non-alphanumeric aside from spaces?

In [26]:
non_alnum_names_malay = [name for name in df_malay['fullname'] if not str.isalnum(name.replace(' ', ''))]
print(len(non_alnum_names_malay))
non_alnum_names_malay[-10:]

200


['Adenan Hj. Satem',
 'P. Balasubramaniam',
 "Abdullah Ma'ayat Shah",
 "Mohd Shahril Sa'ari",
 'E. E. C. Thuraisingham',
 'Ibrahim Ali (Malaysia)',
 'Faradina Mohd. Nadzir',
 'Aril (AF 7)',
 'Sultan Hisamuddin Alam Shah Al-Haj ibni Almarhum Sultan Alaeddin Sulaiman Shah',
 "Ismail Faruqi Asha'ri"]

Again, we have parentheses at the end, so let's remove them:

In [27]:
df_malay['fullname'] = df_malay['fullname'].apply(lambda name: name if name.find('(') == -1 else name[:name.find('(') - 1])
non_alnum_names_malay = [name for name in df_malay['fullname'] if not str.isalnum(name.replace(' ', ''))]
print(len(non_alnum_names_malay))

192


Removing leading and trailing whitespace from names just in case:

In [28]:
df_malay['fullname'] = df_malay['fullname'].apply(str.strip)

Handling duplicates:

In [29]:
# Finding number of duplicates
print(len(df_malay[df_malay.duplicated()]))
df_malay.shape

6


(2914, 1)

In [30]:
# Removing duplicates
df_malay.drop_duplicates(inplace = True, ignore_index = True)
print(np.any(df_malay.duplicated()))
df_malay.shape

False


(2908, 1)

## Feature Engineering

### Indonesian Dataset Feature Engineering

Here are the features we want to include:
1. `alphabet` - which alphabet is being used - unicode library
2. `char_ngrams` - character n-grams (unigram, bigram, trigram, interpolated) - nltk.utils.ngrams()
3. `word_ngrams` - Word n-grams - a limitation is Out of Vocabulary words
4. `a_hat_freq` - binary feature: does it exist in the given name? - if too much work to have EACH accent character as a feature, can combine
5. `a_freq`, `b_freq`, `c_freq` … `z_freq` - frequency of letters
6. `word_length` - length of name (total number of letters in full name)
7. `avg_token_length` - average length of a token (the avg number of characters in each word of the name)
8. `token_length` - token length (how many “words” are in each name) 
9. `transliteration` - transliteration
10. `iso` - ISO language code
11. `period_freq`, `dash_freq`, `apostrophe_freq` - frequency of punctuation


Let's start with the Indonesian dataset.

In [31]:
# 1. alphabet
df_indo['alphabet'] = df_indo['fullname'].apply(lambda name: unicodedata.name(name[0]).split(' ')[0])
print(df_indo['alphabet'].unique())
df_indo.head()

['LATIN']


,fullname,alphabet
0,Supriyadi,LATIN
1,Triyaningsih,LATIN
2,Soerjadi,LATIN
3,Undunsyah,LATIN
4,Soeripto,LATIN


In [40]:
# 6. name_length
df_indo['name_length'] = df_indo['fullname'].apply(len)
df_indo.head()

,fullname,alphabet,name_length,avg_token_length,token_length,iso
0,Supriyadi,LATIN,9,9.0,1,id
1,Triyaningsih,LATIN,12,12.0,1,id
2,Soerjadi,LATIN,8,8.0,1,id
3,Undunsyah,LATIN,9,9.0,1,id
4,Soeripto,LATIN,8,8.0,1,id


In [39]:
# 7. avg_token_length
tokens = df_indo['fullname'].apply(lambda name: name.split(' '))
print(tokens[-5:], '\n')
token_lengths = tokens.apply(lambda token_list: [len(token) for token in token_list])
print(token_lengths[-5:])
df_indo['avg_token_length'] = token_lengths.apply(np.mean)
df_indo.tail()

11241            [Edwel, Yusri, Datuak, Rajo, Gampo, Alam]
11242     [Sultan, Amaluddin, Al, Sani, Perkasa, Alamsyah]
11243          [Prof., Dr., Ir., Antonius, Suwanto,, M.Sc]
11244    [Kanjeng, Raden, Tumengung, Mas, Ariya, Purnam...
11245    [Lord, Kanjeng, Baginda, Atep, Bin, Lord, Dari...
Name: fullname, dtype: object 

11241                      [5, 5, 6, 4, 5, 4]
11242                      [6, 9, 2, 4, 7, 8]
11243                      [5, 3, 3, 8, 8, 4]
11244                  [7, 5, 9, 3, 5, 7, 11]
11245    [4, 7, 7, 4, 3, 4, 4, 6, 4, 7, 7, 8]
Name: fullname, dtype: object


,fullname,alphabet,name_length,avg_token_length,token_length,iso
11241,Edwel Yusri Datuak Rajo Gampo Alam,LATIN,34,4.833333,6,id
11242,Sultan Amaluddin Al Sani Perkasa Alamsyah,LATIN,41,6.000000,6,id
11243,"Prof. Dr. Ir. Antonius Suwanto, M.Sc",LATIN,36,5.166667,6,id
11244,Kanjeng Raden Tumengung Mas Ariya Purnama Hadi...,LATIN,53,6.714286,7,id
11245,Lord Kanjeng Baginda Atep Bin Lord Dari Segala...,LATIN,76,5.416667,12,id


Finding `token_length`:

In [34]:
# 8. token_length
df_indo['token_length'] = df_indo['fullname'].apply(lambda name: len(name.split(' ')))
df_indo.tail()

,fullname,alphabet,name_length,avg_token_length,token_length
11241,Edwel Yusri Datuak Rajo Gampo Alam,LATIN,34,6.0,6
11242,Sultan Amaluddin Al Sani Perkasa Alamsyah,LATIN,41,6.0,6
11243,"Prof. Dr. Ir. Antonius Suwanto, M.Sc",LATIN,36,6.0,6
11244,Kanjeng Raden Tumengung Mas Ariya Purnama Hadi...,LATIN,53,7.0,7
11245,Lord Kanjeng Baginda Atep Bin Lord Dari Segala...,LATIN,76,12.0,12


In [35]:
# 10. iso
df_indo['iso'] = 'id'
df_indo.head()

,fullname,alphabet,name_length,avg_token_length,token_length,iso
0,Supriyadi,LATIN,9,1.0,1,id
1,Triyaningsih,LATIN,12,1.0,1,id
2,Soerjadi,LATIN,8,1.0,1,id
3,Undunsyah,LATIN,9,1.0,1,id
4,Soeripto,LATIN,8,1.0,1,id
